# Imports

In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import re
import torch
import mne
mne.set_log_level(verbose=None)

from tqdm import tqdm
import numpy as np
from matplotlib import pyplot as plt

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K
from tensorflow.keras.losses import BinaryCrossentropy, SparseCategoricalCrossentropy,CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import BinaryAccuracy, AUC, Precision, Recall, CategoricalAccuracy
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

ImportError: cannot import name 'plot_confusion_matrix' from 'sklearn.metrics' (/home/huangcb/anaconda3/envs/EEGWave/lib/python3.8/site-packages/sklearn/metrics/__init__.py)

# Mode

In [186]:
MODE = "CLASSIC"
# MODE = "DOUBLE" # we double the size of the training subset by mixing the same amount of generated signals as the number in the original subset (6)
# MODE = "PRETRAIN" # pre-train EEGNet on the generated data and then re-train this initialized model with the real signals (7)

DATA = "BCICIV"

# Global variables

In [187]:
BCICIV2A_PATH = "/kaggle/input/eegpreprocessed/pt_bciciv_2a"
BCICIV2B_PATH = "/kaggle/input/eegpreprocessed/pt_bciciv_2b"
BCICIV2A_AUG_PATH = "/kaggle/input/eegpreprocessed/pt_bciciv_2a_aug"
BCICIV2B_AUG_PATH = "/kaggle/input/eegpreprocessed/pt_bciciv_2b_aug"
N_TENSORS = -1
 
if DATA == "BCICIV_2A":
    PATH = BCICIV2A_PATH
    if MODE == "AUGMENTED": PATH_AUG = BCICIV2A_AUG_PATH
    N_CLASSES = 4
    classnames = ["0", "1", "2", "3"]
    N_CHANNELS = 25
    N_SAMPLES = 448

else DATA == "BCICIV_2B":
    PATH = BCICIV2B_PATH
    if MODE == "AUGMENTED": PATH_AUG = BCICIV2B_AUG_PATH
    N_CLASSES = 2
    classnames = ["0", "1"]
    N_CHANNELS = 3
    N_SAMPLES = 448


BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = BATCH_SIZE * 2 

FIRST_SPLIT = 0.12
SECOND_SPLIT = 0.15

EPOCHS = 200
LEARNING_RATE = 1e-3
MIN_DELTA = 1e-5
PATIENCE = 20

# Preprocessing data functions

In [188]:
# Already sampled / preprocessed
def load_pt_data(path):
    cpt = 0
    data, labels, users = [], [], []
    for filename in tqdm(os.listdir(path)):
        if cpt >= N_TENSORS and N_TENSORS != -1:
            break
        f = os.path.join(path, filename)
        x, y, u = torch.load(f)
        data.append(x)
        labels.append(y)
        users.append(u)
        cpt += 1
    return np.array(data), np.array(labels), np.array(users)

def split_data(preprocessed_data, final_labels, final_user_idx):
    split = StratifiedShuffleSplit(n_splits=2, test_size=FIRST_SPLIT, random_state=0)
    for x_index, test_index in split.split(preprocessed_data, final_user_idx):
        X, X_test = preprocessed_data[x_index], preprocessed_data[test_index]
        y, y_test = final_labels[x_index], final_labels[test_index]
        users, users_test = final_user_idx[x_index], final_user_idx[test_index]
#     print(X.shape, X_test.shape, y.shape, y_test.shape)
#     print(np.unique(users, return_counts=True), np.unique(users_test, return_counts=True))
    split = StratifiedShuffleSplit(n_splits=2, test_size=SECOND_SPLIT, random_state=0)
    for train_index, valid_index in split.split(X, users):
        X_train, X_valid = X[train_index], X[valid_index] 
        y_train, y_valid = y[train_index], y[valid_index]
        users_train, users_valid = users[train_index], users[valid_index]
#     print(X_train.shape, X_valid.shape, y_train.shape, y_valid.shape)
#     print(np.unique(y_train, return_counts=True), np.unique(y_test, return_counts=True))
#     print(np.unique(users_train, return_counts=True))
#     print(np.unique(users_valid, return_counts=True))
#     print(np.unique(users_test, return_counts=True))
    return X_train, y_train, X_valid, y_valid, X_test, y_test

def one_hot_data(y_train, y_valid, y_test):
    if DATA == "BCICIV_2B":
        y_train = np.where(y_train==34, 0, 1)
        y_valid = np.where(y_valid==34, 0, 1)
        y_test = np.where(y_test==34, 0, 1)
    else:
        y_train = y_train - 1
        y_valid = y_valid - 1
        y_test = y_test - 1
    y_train = tf.one_hot(y_train, depth=N_CLASSES, axis=0)
    y_valid = tf.one_hot(y_valid, depth=N_CLASSES, axis=0)
    y_test = tf.one_hot(y_test, depth=N_CLASSES, axis=0)
    return tf.transpose(y_train), tf.transpose(y_valid), tf.transpose(y_test)

def make_tensorflow_dataset(X_train, y_train, X_valid, y_valid, X_test, y_test):
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    valid_dataset = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))
    test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
    train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
    valid_dataset = valid_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
    test_dataset = test_dataset.batch(BATCH_SIZE)
    return train_dataset, valid_dataset, test_dataset

# Preprocessing data

In [189]:
if MODE == "CLASSIC":
    data = load_pt_data(PATH)
    X_train, y_train, X_valid, y_valid, X_test, y_test = split_data(data[0], data[1], data[2])
    y_train, y_valid, y_test = one_hot_data(y_train, y_valid, y_test)
    train_dataset, valid_dataset, test_dataset = make_tensorflow_dataset(X_train, 
                                                                         y_train, 
                                                                         X_valid, 
                                                                         y_valid, 
                                                                         X_test, 
                                                                         y_test)
elif MODE == "DOUBLE":
    data_base = load_pt_data(PATH)
    data_aug = load_pt_data(PATH_AUG)
    data = data_base + data_aug
    X_train, y_train, X_valid, y_valid, X_test, y_test = split_data(data[0], data[1], data[2])
    y_train, y_valid, y_test = one_hot_data(y_train, y_valid, y_test)
    train_dataset, valid_dataset, test_dataset = make_tensorflow_dataset(X_train, 
                                                                         y_train, 
                                                                         X_valid, 
                                                                         y_valid, 
                                                                         X_test, 
                                                                         y_test)
elif MODE == "PRETRAIN":
    data_base = load_pt_data(PATH)
    X_train, y_train, X_valid, y_valid, X_test, y_test = split_data(data_base[0], data_base[1], data_base[2])
    y_train, y_valid, y_test = one_hot_data(y_train, y_valid, y_test)
    train_dataset, valid_dataset, test_dataset = make_tensorflow_dataset(X_train, 
                                                                         y_train, 
                                                                         X_valid, 
                                                                         y_valid, 
                                                                         X_test, 
                                                                         y_test)
    data_aug = load_pt_data(PATH_AUG)
    X_train_aug, y_train, X_valid, y_valid, X_test, y_test = split_data(data_aug[0], data_aug[1], data_aug[2])
    y_train_aug, y_valid_aug, y_test_aug = one_hot_data(y_train_aug, y_valid_aug, y_test_aug)
    train_dataset_aug, valid_dataset_aug, test_dataset_aug = make_tensorflow_dataset(X_train_aug, 
                                                                                     y_train_aug, 
                                                                                     X_valid_aug, 
                                                                                     y_valid_aug, 
                                                                                     X_test_aug, 
                                                                                     y_test_aug)

100%|██████████| 2592/2592 [00:03<00:00, 730.49it/s]


In [190]:
print(np.unique(y_train.numpy().argmax(axis=1), return_counts=True))
print(np.unique(y_valid.numpy().argmax(axis=1), return_counts=True))
print(np.unique(y_test.numpy().argmax(axis=1), return_counts=True))
print(X_train.min(), X_train.max(), X_train.mean(), X_train.std())

(array([0, 1, 2, 3]), array([481, 490, 484, 483]))
(array([0, 1, 2, 3]), array([94, 77, 86, 85]))
(array([0, 1, 2, 3]), array([73, 81, 78, 80]))
-1.0 1.0 0.019062353726686045 0.34398400833170356


# Model functions

In [191]:
def get_callbacks():
    return [
        # ModelCheckpoint(
        #     "best_eegnet.h5", save_best_only=True, monitor="val_loss"
        # ),
        # ReduceLROnPlateau(
        #     monitor="val_top_k_categorical_accuracy",
        #     factor=0.2,
        #     patience=2,
        #     min_lr=0.000001,
        # ),
        EarlyStopping(
            monitor="val_loss",
            min_delta=MIN_DELTA,
            patience=PATIENCE,
            verbose=1,
            mode="auto",
            baseline=None,
            restore_best_weights=True,
        ),
    ]

# https://arxiv.org/pdf/1611.08024.pdf
def EEGNet(n_classes, n_channels=64, n_samples=128, kernel_length=64, n_filters1=8, 
           n_filters2=16, depth_multiplier=2, norm_rate=0.25, dropout_rate=0.5, 
           dropoutType="Dropout"):

    if dropoutType == "SpatialDropout2D":
        dropoutType=SpatialDropout2D
    elif dropoutType == "Dropout":
        dropoutType=Dropout

    inputs = Input(shape=(n_channels, n_samples, 1))

    block1 = Conv2D(n_filters1, (1, kernel_length), padding="same", input_shape=(n_channels, n_samples, 1), use_bias=False)(inputs)
    block1 = BatchNormalization()(block1)
    block1 = DepthwiseConv2D((n_channels, 1), use_bias=False, depth_multiplier=depth_multiplier, depthwise_constraint=max_norm(1.0))(block1)
    block1 = BatchNormalization()(block1)
    block1 = Activation("elu")(block1)
    block1 = AveragePooling2D((1, 4))(block1)
    block1 = dropoutType(dropout_rate)(block1)

    block2 = SeparableConv2D(n_filters2, (1, 16), use_bias=False, padding="same")(block1)
    block2 = BatchNormalization()(block2)
    block2 = Activation("elu")(block2)
    block2 = AveragePooling2D((1, 8))(block2)
    block2 = dropoutType(dropout_rate)(block2)
    block2 = Flatten(name="flatten")(block2)

    classifier = Dense(n_classes, name="dense", kernel_constraint=max_norm(norm_rate))(block2)
    classifier = Activation("softmax", name="softmax")(classifier)

    model = Model(inputs=inputs, outputs=classifier) 

    optimizer = Adam(amsgrad=True, learning_rate=LEARNING_RATE)
    
    if DATA == "BCICIV_2A":
        loss = CategoricalCrossentropy()
        acc = CategoricalAccuracy()
    elif DATA == "BCICIV_2B":
        loss = BinaryCrossentropy()
        acc = BinaryAccuracy()
        
    model.compile(
            optimizer=optimizer,
            loss=loss,
            metrics=[
                acc,
    #             AUC(),
    #             Precision(),
    #             Recall(),
            ],
        )

    model.summary()

    return model

def plot_history_metrics(history):
    total_plots = len(history.history)
    cols = total_plots // 2

    rows = total_plots // cols

    if total_plots % cols != 0:
        rows += 1

    pos = range(1, total_plots + 1)
    plt.figure(figsize=(15, 10))
    for i, (key, value) in enumerate(history.history.items()):
        plt.subplot(rows, cols, pos[i])
        plt.plot(range(len(value)), value)
        plt.title(str(key))
    plt.show()

# Training

In [ ]:
classes = np.unique(y_train.numpy().argmax(axis=1))
class_weights = class_weight.compute_class_weight(class_weight="balanced", classes=classes, y=y_train.numpy().argmax(axis=1))
class_weights = dict(zip(classes, class_weights))
print(class_weights)

In [ ]:
if MODE == "CLASSIC" or MODE == "DOUBLE":
    model = EEGNet(n_classes=N_CLASSES, n_channels=N_CHANNELS, n_samples=N_SAMPLES)
    history = model.fit(
        train_dataset,
        epochs=EPOCHS,
        callbacks=get_callbacks(),
        validation_data=valid_dataset,
        class_weight=class_weights,
    )
elif MODE == "PRETRAIN":
    model = EEGNet(n_classes=N_CLASSES, n_channels=N_CHANNELS, n_samples=N_SAMPLES)  
    pretrain_history = model.fit(
        train_dataset_aug,
        epochs=EPOCHS,
        callbacks=get_callbacks(),
        validation_data=valid_dataset_aug,
        class_weight=class_weights_aug,
    )
    final_history = model.fit(
        train_dataset,
        epochs=EPOCHS,
        callbacks=get_callbacks(),
        validation_data=valid_dataset,
        class_weight=class_weights_aug,
    )

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 25, 448, 1)]      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 25, 448, 8)        512       
_________________________________________________________________
batch_normalization_33 (Batc (None, 25, 448, 8)        32        
_________________________________________________________________
depthwise_conv2d_11 (Depthwi (None, 1, 448, 16)        400       
_________________________________________________________________
batch_normalization_34 (Batc (None, 1, 448, 16)        64        
_________________________________________________________________
activation_22 (Activation)   (None, 1, 448, 16)        0         
_________________________________________________________________
average_pooling2d_22 (Averag (None, 1, 112, 16)        0  

# Plotting learning curves

In [ ]:
if MODE == "CLASSIC" or MODE == "DOUBLE":
    plot_history_metrics(history)
elif MODE == "PRETRAIN":
    plot_history_metrics(pretrain_history)
    plot_history_metrics(final_history)

# Final evaluation on test set

In [ ]:
# Trick to use plot function from sklearn
class estimator:
    _estimator_type = ""
    classes_=[]
    def __init__(self, model, classes):
        self.model = model
        self._estimator_type = "classifier"
        self.classes_ = classes
    def predict(self, X):
        y_prob= self.model.predict(X)
        y_pred = y_prob.argmax(axis=1)
        return y_pred

classifier = estimator(model, classnames)

In [ ]:
color = "white"

matrix = plot_confusion_matrix(classifier, X_train, y_train.numpy().argmax(axis=1), cmap=plt.cm.Blues)
matrix.ax_.set_title("Train confusion matrix", color=color)
plt.xlabel("Predicted Label", color=color)
plt.ylabel("True Label", color=color)
plt.gcf().axes[0].tick_params(colors=color)
plt.gcf().axes[1].tick_params(colors=color)
plt.show()

matrix = plot_confusion_matrix(classifier, X_valid, y_valid.numpy().argmax(axis=1), cmap=plt.cm.Blues)
matrix.ax_.set_title("Valid confusion matrix", color=color)
plt.xlabel("Predicted Label", color=color)
plt.ylabel("True Label", color=color)
plt.gcf().axes[0].tick_params(colors=color)
plt.gcf().axes[1].tick_params(colors=color)
plt.show()

matrix = plot_confusion_matrix(classifier, X_test, y_test.numpy().argmax(axis=1), cmap=plt.cm.Blues)
matrix.ax_.set_title("Test confusion matrix", color=color)
plt.xlabel("Predicted Label", color=color)
plt.ylabel("True Label", color=color)
plt.gcf().axes[0].tick_params(colors=color)
plt.gcf().axes[1].tick_params(colors=color)
plt.show()

In [ ]:
y_pred = model.predict(X_test).argmax(axis=1)
y_true = y_test.numpy().argmax(axis=1)
print(classification_report(y_true, y_pred))